# Proyecto 7: Deploy de sistema de recomendación con Watson

En este proyecto llevaremos a cabo la puesta en producción del modelo entrenado en el proyecto 5. Es decir, lo subirmos la nube de IBM y utilizando llamados a la API de Watson tendremos acceso a él para realizar predicciones.

In [41]:
import warnings
warnings.filterwarnings("ignore")
import sklearn
from sklearn.datasets import load_files
#moviedir = r'./dataset/movie_reviews' 
moviedir = 'C://Users//tnlocaladmin//OneDrive//Documentos//DS//Proyectos//Entrega 5//dataset//movie_reviews'
movie_reviews = load_files(moviedir, shuffle=True)

In [42]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    movie_reviews.data, movie_reviews.target, test_size = 0.20, stratify=movie_reviews.target, random_state = 12)

In [43]:
from sklearn.externals import joblib
eclf = joblib.load('sentiment.pkl') 

IBM Watson

**1) Cargá** la biblioteca `WatsonMachineLearningAPIClient`

In [44]:
#!pip install watson-machine-learning-client
#!Pip install --upgrade watson-developer-cloud

In [45]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

**2) Creá** variable con las credenciales que necesita `Watson`. Ellas son: `url, access_key, username, password e instance_id`

In [46]:
cred = {
  "apikey": "PM9-fU8i8gQlsEUwRNYiw65eI_DUduREHg7QZrqJPKWL",
  "iam_apikey_description": "Auto-generated for key 56d1aa85-5617-432a-b344-629c57617073",
  "iam_apikey_name": "Service credentials-1",
  "iam_role_crn": "crn:v1:bluemix:public:iam::::serviceRole:Manager",
  "iam_serviceid_crn": "crn:v1:bluemix:public:iam-identity::a/6e95f0bb93b1425eb5b2b56281dd121e::serviceid:ServiceId-91816014-88ec-4e06-89e7-77547f385f84",
  "instance_id": "6fe030c5-dc7a-4365-90ae-0ef5ae8f047e",
  "password": "f1e9adbd-ee10-42be-8081-648d0eb5db29",
  "url": "https://us-south.ml.cloud.ibm.com",
  "username": "56d1aa85-5617-432a-b344-629c57617073"
}


**3) Declará** la variable `client` y guardá en ella al objeto `WatsonMachineLearningAPIClient` con las credenciales como parámetro

In [47]:
cliente = WatsonMachineLearningAPIClient(cred)

**4) Creá** una variable que guarde las propiedades del modelo. Datos del autor y nombre del proyecto.

In [48]:
model_props = {cliente.repository.ModelMetaNames.AUTHOR_NAME: "Pablo Espinosa",
                        cliente.repository.ModelMetaNames.NAME: "Procesamiento de Lenguaje Natural"}

**5) Hacé** un pipeline que contenga como primer paso a un `TfidfVectorizer` y como segundo paso, al mejor modelo que hayas obtenido en el proyecto 5. **Entrená** con este pipeline.

In [49]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    movie_reviews.data, movie_reviews.target, test_size = 0.20, stratify=movie_reviews.target, random_state = 8)

In [50]:
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords

stops=set(stopwords.words('english'))

vect = TfidfVectorizer(stop_words=stops, tokenizer = nltk.word_tokenize)

pipeline = make_pipeline(vect, eclf)
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('tfidfvectorizer', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_i...rs=500, random_state=0))],
         flatten_transform=None, n_jobs=1, voting='hard', weights=None))])

**6) Subí** al modelo a IBM Cloud usando `client.repository.store_model` con los parámetros correctos.

In [51]:
published_model = cliente.repository.store_model(model=pipeline, 
                                                meta_props=model_props, 
                                                training_data=X_train, 
                                                training_target=y_train)

**7) Obtené** el `uid` del modelo y guardalo en una variable.

In [52]:
published_model_uid = cliente.repository.get_model_uid(published_model)

In [53]:
models_details = cliente.repository.list_models()

------------------------------------  ---------------------------------  ------------------------  -----------------
GUID                                  NAME                               CREATED                   FRAMEWORK
c5b346b5-a1f6-45a2-bdbd-5bebfd7526d5  Procesamiento de Lenguaje Natural  2019-06-05T20:02:32.918Z  scikit-learn-0.19
5019ace4-7ac7-4c61-bf19-f82e64066e72  Procesamiento de Lenguaje Natural  2019-06-05T19:54:02.765Z  scikit-learn-0.19
98c1ce44-778d-4611-9724-7272eb5030de  Procesamiento de Lenguaje Natural  2019-06-05T19:50:24.369Z  scikit-learn-0.19
ffa2b500-c9e0-4fff-954f-90795ab03828  Procesamiento de Lenguaje Natural  2019-06-05T19:33:14.315Z  scikit-learn-0.19
761bb66e-bc10-443f-bbcc-f98f02bf6e39  Procesamiento de Lenguaje Natural  2019-06-05T18:58:43.886Z  scikit-learn-0.19
------------------------------------  ---------------------------------  ------------------------  -----------------


**8) Cargá** el modelo basado en su `uid` y utilizalo para realizar la predicción sobre el conjunto de test

In [54]:
loaded_model = cliente.repository.load(published_model_uid)

In [55]:
test_predictions = loaded_model.predict(X_test)

**9) Mostrar** el `classification_report` obtenido

In [56]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report

print("roc_auc score: ", roc_auc_score(y_test, test_predictions))
print(classification_report(y_test, test_predictions))

roc_auc score:  0.8475
             precision    recall  f1-score   support

          0       0.81      0.91      0.86       200
          1       0.89      0.79      0.84       200

avg / total       0.85      0.85      0.85       400

